<a href="https://colab.research.google.com/github/eGamez01/cbm_plus/blob/main/cbm_plus_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/eGamez01/cbm_plus.git # Replace with the actual URL of your repository

In [ ]:
# -----------------------------------------------------------
# 0. Imports
# -----------------------------------------------------------
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import weibull_min, gamma, kstest

# -----------------------------------------------------------
# 1. Point to your local folder  (edit as needed)
# -----------------------------------------------------------
DATA_DIR = Path(r"/content/cbm_plus/data/CMaps")          # <— change me
SCENARIO  = "FD001"                       # "FD001" | "FD002" | "FD003" | "FD004"
train_path = DATA_DIR / f"train_{SCENARIO}.txt"

# Column layout for every C-MAPSS file: 2 ID cols + 3 op settings + 21 sensors
cols = (["unit", "cycle"] +
        [f"op_setting_{i}" for i in range(1, 4)] +
        [f"s{i}"           for i in range(1, 22)])

# -----------------------------------------------------------
# 2. Load the training data
# -----------------------------------------------------------
train_df = pd.read_csv(train_path, delim_whitespace=True, header=None, names=cols)
print(f"Loaded {train_df.shape[0]:,} rows from {train_path.name}")

# -----------------------------------------------------------
# 3. Derive each engine’s life (cycles-to-failure)
# -----------------------------------------------------------
lifes = train_df.groupby("unit")["cycle"].max().to_numpy()
print(f"{lifes.size} engines – life range {lifes.min()}-{lifes.max()} cycles")

# -----------------------------------------------------------
# 4a. Weibull (2-param) fit
# -----------------------------------------------------------
beta, _, eta = weibull_min.fit(lifes, floc=0)        # shape, loc=0, scale
# 4b. Gamma (2-param) fit
k_gamma, _, theta_gamma = gamma.fit(lifes, floc=0)   # shape, loc=0, scale

# -----------------------------------------------------------
# 5. Optional: log-likelihood & AIC for quick comparison
# -----------------------------------------------------------
ll_weib  = weibull_min.logpdf(lifes, beta,      0, eta        ).sum()
ll_gamma = gamma      .logpdf(lifes, k_gamma,   0, theta_gamma).sum()
aic_weib  = 2*2 - 2*ll_weib       # 2 parameters (β,η)
aic_gamma = 2*2 - 2*ll_gamma      # 2 parameters (k,θ)

print("\n— Parameter fits —")
print(f"Weibull  : β={beta:.3f},  η={eta:.3f}")
print(f"Gamma    : k={k_gamma:.3f}, θ={theta_gamma:.3f}")
print("\n— Model scores —")
print(f"Weibull  LL={ll_weib:.2f},  AIC={aic_weib:.2f}")
print(f"Gamma    LL={ll_gamma:.2f}, AIC={aic_gamma:.2f}")

# -----------------------------------------------------------
# 6. Build PDF & Reliability curves for overlay
# -----------------------------------------------------------
t = np.linspace(0, lifes.max()*1.1, 400)

pdf_weib  = weibull_min.pdf(t,  beta,    0, eta)
pdf_gamma = gamma.pdf(       t, k_gamma, 0, theta_gamma)

rel_weib  = 1 - weibull_min.cdf(t, beta,    0, eta)
rel_gamma = 1 - gamma.cdf(       t, k_gamma, 0, theta_gamma)

# -----------------------------------------------------------
# 7. Plot PDF over histogram
# -----------------------------------------------------------
plt.figure(figsize=(9,5))
plt.hist(lifes, bins=12, density=True, alpha=.35, label="Empirical lives")
plt.plot(t, pdf_weib,  lw=2, label="Weibull PDF")
plt.plot(t, pdf_gamma, lw=2, label="Gamma PDF", ls="--")
plt.xlabel("Cycles-to-failure");  plt.ylabel("Probability density")
plt.title(f"Weibull vs Gamma – C-MAPSS {SCENARIO}")
plt.legend()
plt.show()

# -----------------------------------------------------------
# 8. Plot reliability curves on same axes
# -----------------------------------------------------------
plt.figure(figsize=(9,5))
plt.plot(t, rel_weib,  lw=2, label="Weibull Reliability")
plt.plot(t, rel_gamma, lw=2, label="Gamma Reliability", ls="--")
plt.xlabel("Cycles");  plt.ylabel("Reliability  R(t)")
plt.title("Reliability curves comparison")
plt.ylim(0, 1.05); plt.legend()
plt.show()